In [ ]:
%load_ext autoreload
%autoreload 2

This is to demonstrate how to use the `s1-enumerator` to get a full time series of GUNWs.

We are going basically take each month in acceptable date range and increment by a month and make sure the temporal window is large enough to ensure connectivity across data gaps.

# Parameters

This is what the operator is going to have to change. Will provide some comments.

In [ ]:
# toggle user-controlled parameters here
import datetime
import json

# product cutline
aoi_shapefile = '../aois/CA_pathNumber115.geojson'
# load AO geojson and strip variables
with open(aoi_shapefile, 'r') as file:
    data = json.load(file)
    json_keys = data['features'][0]['properties'].keys()
    # assign local variables from dict
    locals().update(data['features'][0]['properties'])

# Override metadata keys
# Warning, toggle with caution
# Only to be used if you are testing and intentionally updating AO geojsons
update_AO = True

### Spatial coverage constraint parameter 'azimuth_mismatch'
# The merged SLC area over the AOI is allowed to be smaller by 'azimuth_mismatch' x swath width (i.e. 250km)
if 'azimuth_mismatch' not in json_keys or update_AO:
    azimuth_mismatch = 5 # adjust as necessary
    data['features'][0]['properties']['azimuth_mismatch'] = azimuth_mismatch

# Specify deployment URL
#deploy_url = 'https://hyp3-tibet-jpl.asf.alaska.edu' #for Tibet
deploy_url = 'https://hyp3-nisar-jpl.asf.alaska.edu' #for access
data['features'][0]['properties']['deploy_url'] = deploy_url

# Number of nearest neighbors
if 'num_neighbors' not in json_keys or update_AO:
    num_neighbors = 2 # adjust as necessary
    data['features'][0]['properties']['num_neighbors'] = num_neighbors

#set temporal parameters
if 'min_days_backward' not in json_keys or update_AO:
    min_days_backward = 0
    data['features'][0]['properties']['min_days_backward'] = min_days_backward
today = datetime.datetime.now()
# Earliest year for reference frames
START_YEAR = 2014
# Latest year for reference frames
END_YEAR = today.year
YEARS_OF_INTEREST = list(range(START_YEAR,END_YEAR+1))
# Adjust depending on seasonality
# For annual IFGs, select a single months of interest and you will get what you want.
if 'month_range_lower' not in json_keys or update_AO:
    month_range_lower = 1 # adjust as necessary
    data['features'][0]['properties']['month_range_lower'] = month_range_lower
if 'month_range_upper' not in json_keys or update_AO:
    month_range_upper = 12 # adjust as necessary
    data['features'][0]['properties']['month_range_upper'] = month_range_upper
MONTHS_OF_INTEREST = list(range(month_range_lower,month_range_upper+1))


############################################################################################################
## OPTIONAL set temporal sampling parameters

# Temporal sampling invervals
if 'min_days_backward_timesubset' not in json_keys or update_AO:
    # Specify as many temporal sampling intervals as desired (e.g. 90 (days), 180 (days) = semiannual, 365 (days) = annual, etc.)
    min_days_backward_timesubset = []
    if min_days_backward_timesubset != []:
        data['features'][0]['properties']['min_days_backward_timesubset'] = ','.join(map(str, min_days_backward_timesubset))
if 'min_days_backward_timesubset' in json_keys:
    min_days_backward_timesubset = [int(s) for s in data['features'][0]['properties']['min_days_backward_timesubset'].split(',')]
# apply temporal window to all temporal sampling intervals (hardcoded to 60 days)
temporal_window_days_timesubset = 60
min_days_backward_timesubset = [i - round(temporal_window_days_timesubset/2) for i in min_days_backward_timesubset]
if any(x<1 for x in min_days_backward_timesubset):
    raise Exception("Your specified 'min_days_backward_timesubset' input is too small relative to"
                    "your specified 'temporal_window_days_timesubset' value. Adjust accordingly")

# Nearest neighbor sampling
if 'num_neighbors_timesubset' not in json_keys or update_AO:
    # Specify corresponding nearest neighbor sampling for each temporal sampling interval (by default (n-)1)
    num_neighbors_timesubset = []
    if num_neighbors_timesubset != []:
        data['features'][0]['properties']['num_neighbors_timesubset'] = ','.join(map(str, num_neighbors_timesubset))
if 'num_neighbors_timesubset' in json_keys:
    num_neighbors_timesubset = [int(s) for s in data['features'][0]['properties']['num_neighbors_timesubset'].split(',')]

if len (num_neighbors_timesubset) != len(min_days_backward_timesubset):
    raise Exception("Specified number of temporal sampling intervals DO NOT match specified nearest neighbor sampling")
############################################################################################################


# Define job-name
if 'job_name' not in json_keys or update_AO:
    job_name = aoi_shapefile.split('/')[-1].split('.')[0].split('pathNumber')
    job_name = ''.join(job_name)
    job_name = job_name[-20:]
    data['features'][0]['properties']['job_name'] = job_name
# product directory
prod_dir = job_name

# if operator variables do not exist, set and populate geojson with them
with open(aoi_shapefile, 'w') as file:
    json.dump(data, file)

In [ ]:
from s1_enumerator import get_aoi_dataframe,  distill_all_pairs, enumerate_ifgs, get_s1_coverage_tiles, enumerate_ifgs_from_stack, get_s1_stack_by_dataframe
import concurrent
from rasterio.crs import CRS
from s1_enumerator import duplicate_gunw_found
from tqdm import tqdm
from shapely.geometry import shape
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from dateutil.relativedelta import relativedelta
import networkx as nx
import boto3
import hyp3_sdk
import copy

from deploytools.parse_tools import (shapefile_area, 
                             continuous_time, 
                             minimum_overlap_query, 
                             pair_spatial_check)

In [ ]:
df_aoi = gpd.read_file(aoi_shapefile)
aoi = df_aoi.geometry.unary_union
aoi

Currently, there is a lot of data in each of the rows above. We really only need the AOI `geometry` and the `path_number`.

In [ ]:
path_numbers = df_aoi.path_number.unique().tolist()

# Generate a stack

Using all the tiles that are needed to cover the AOI we make a geometric query based on the frame. We now include only the path we are interested in.

In [ ]:
path_dict = {}
path_dict['pathNumber'] = str(path_numbers[0])
aoi_geometry = pd.DataFrame([path_dict])
aoi_geometry = gpd.GeoDataFrame(aoi_geometry, geometry=[shape(aoi)], crs=CRS.from_epsg(4326))
aoi_geometry['pathNumber'] = aoi_geometry['pathNumber'].astype(int)

In [ ]:
df_stack = get_s1_stack_by_dataframe(aoi_geometry,
                                     path_numbers=path_numbers)

In [ ]:
f'We have {df_stack.shape[0]} frames in our stack'

In [ ]:
fig, ax = plt.subplots()

df_stack.plot(ax=ax, alpha=.5, color='green', label='Frames interesecting tile')
df_aoi.exterior.plot(color='black', ax=ax, label='AOI')
plt.legend()

Note, we now see the frames cover the entire AOI as we expect.

First remove all scenes that do not produce spatiotemporally contiguous pairs and not meet specified intersection threshold

In [ ]:
df_stack, df_stack_dict, gap_scenes_dict, rejected_scenes_dict = minimum_overlap_query(df_stack, aoi, azimuth_mismatch=azimuth_mismatch)

In [ ]:
f'We have {df_stack.shape[0]} frames in our stack'

Plot acquisitions that aren't continuous (i.e. have gaps)

In [ ]:
if not gap_scenes_dict.empty:
    gap_scenes_dict =  gap_scenes_dict.sort_values(by=['start_date'])
    for index, row in gap_scenes_dict.iterrows():
        fig, ax = plt.subplots()
        p = gpd.GeoSeries(row['geometry'])
        p.exterior.plot(color='black', ax=ax, label=row['start_date_str'][0])
        df_aoi.exterior.plot(color='red', ax=ax, label='AOI')
        plt.legend()
        plt.show

Plot all mosaicked acquisitions that were rejected for not meeting user-specified spatial constraints

In [ ]:
if not rejected_scenes_dict.empty:
    rejected_scenes_dict =  rejected_scenes_dict.sort_values(by=['start_date'])
    fig, ax = plt.subplots()
    for index, row in rejected_scenes_dict.iterrows():
        p = gpd.GeoSeries(row['geometry'])
        p.exterior.plot(color='black', ax=ax)
    df_aoi.exterior.plot(color='red', ax=ax, label='AOI')
    plt.legend()

Plot each individual mosaicked acquisitions that were rejected for not meeting user-specified spatial constraints

In [ ]:
if not rejected_scenes_dict.empty:
    for index, row in rejected_scenes_dict.iterrows():
        fig, ax = plt.subplots()
        p = gpd.GeoSeries(row['geometry'])
        p.exterior.plot(color='black', ax=ax, label=row['start_date_str'][0])
        df_aoi.exterior.plot(color='red', ax=ax, label='AOI')
        plt.legend()
        plt.show

Plot all mosaicked acquisitions that meet user-defined spatial coverage

In [ ]:
fig, ax = plt.subplots()

for index, row in df_stack_dict.iterrows():
    p = gpd.GeoSeries(row['geometry'])
    p.exterior.plot(color='black', ax=ax)
df_aoi.exterior.plot(color='red', ax=ax, label='AOI')
plt.legend()

Next, we filter the stack by month to ensure we only have SLCs we need.

In [ ]:
df_stack_month = df_stack[df_stack.start_date.dt.month.isin(MONTHS_OF_INTEREST)]
df_stack_month = df_stack_month[df_stack_month.start_date.dt.year.isin(YEARS_OF_INTEREST)]

We will create a list of ```min_reference_dates``` in descending order starting with the most recent date from the SLC stack ```df_stack_month``` as the start date.

In [ ]:
min_reference_dates = sorted(df_stack_month['startTime'].to_list())
min_reference_dates = sorted(list(set([i.replace(hour=0, minute=0, second=0) for i in min_reference_dates])), reverse = True)

We can now enumerate the SLC pairs that will produce the interferograms (GUNWs) based on initially defined parameters that are exposed at the top-level of this jupyter notebook.

In [ ]:
ifg_pairs = []
temporal_window_days = 365*3

# Avoid duplicate reference scenes (i.e. extra neighbors than intended)
track_ref_dates = []
for min_ref_date in tqdm(min_reference_dates):
    temp = enumerate_ifgs_from_stack(df_stack_month,
                                     aoi,
                                     min_ref_date,
                                     enumeration_type='tile', # options are 'tile' and 'path'. 'path' processes multiple references simultaneously
                                     min_days_backward=min_days_backward,
                                     num_neighbors_ref=1,
                                     num_neighbors_sec=num_neighbors,
                                     temporal_window_days=temporal_window_days,
                                     min_tile_aoi_overlap_km2=.1,#Minimum reference tile overlap of AOI in km2
                                     min_ref_tile_overlap_perc=.1,#Relative overlap of secondary frames over reference frame
                                     minimum_ifg_area_km2=0.1,#The minimum overlap of reference and secondary in km2
                                     minimum_path_intersection_km2=.1,#Overlap of common track union with respect to AOI in km2
                                         )
    if temp != []:
        iter_key = temp[0]['reference']['start_date'].keys()[0]
        iter_references_scenes = [temp[0]['reference']['start_date'][iter_key]]
        if not any(x in iter_references_scenes for x in track_ref_dates):
            track_ref_dates.extend(iter_references_scenes)            
            ifg_pairs += temp

OPTIONAL densify network with temporal sampling parameters

In [ ]:
# Densify network with specified temporal sampling

# Avoid duplicate reference scenes (i.e. extra neighbors than intended)
if min_days_backward_timesubset != []:
    for t_ind,t_interval in enumerate(min_days_backward_timesubset):
        track_ref_dates = []
        for min_ref_date in tqdm(min_reference_dates):
            temp = enumerate_ifgs_from_stack(df_stack_month,
                                             aoi,
                                             min_ref_date,
                                             enumeration_type='tile', # options are 'tile' and 'path'. 'path' processes multiple references simultaneously
                                             min_days_backward=t_interval,
                                             num_neighbors_ref=1,
                                             num_neighbors_sec=num_neighbors_timesubset[t_ind],
                                             temporal_window_days=temporal_window_days_timesubset,
                                             min_tile_aoi_overlap_km2=.1,#Minimum reference tile overlap of AOI in km2
                                             min_ref_tile_overlap_perc=.1,#Relative overlap of secondary frames over reference frame
                                             minimum_ifg_area_km2=0.1,#The minimum overlap of reference and secondary in km2
                                             minimum_path_intersection_km2=.1,#Overlap of common track union with respect to AOI in km2
                                                 )
            if temp != []:
                iter_key = temp[0]['reference']['start_date'].keys()[0]
                iter_references_scenes = [temp[0]['reference']['start_date'][iter_key]]
                if not any(x in iter_references_scenes for x in track_ref_dates):
                    track_ref_dates.extend(iter_references_scenes)            
                    ifg_pairs += temp

In [ ]:
f'The number of GUNWs (likely lots of duplicates) is {len(ifg_pairs)}'

# Get Dataframe

In [ ]:
df_pairs = distill_all_pairs(ifg_pairs)
f"# of GUNWs: ' {df_pairs.shape[0]}"

# Deduplication Pt. 1

A `GUNW` is uniquely determined by the reference and secondary IDs. We contanenate these sorted lists and generate a lossy hash to deduplicate products we may have introduced from the enumeration above.

In [ ]:
import hashlib
import json


def get_gunw_hash_id(reference_ids: list, secondary_ids: list) -> str:
    all_ids = json.dumps([' '.join(sorted(reference_ids)),
                          ' '.join(sorted(secondary_ids))
                          ]).encode('utf8')
    hash_id = hashlib.md5(all_ids).hexdigest()
    return hash_id

def hasher(row):
    return get_gunw_hash_id(row['reference'], row['secondary'])

In [ ]:
df_pairs['hash_id'] = df_pairs.apply(hasher, axis=1)
f"# of duplicated entries: {df_pairs.duplicated(subset=['hash_id']).sum()}"

In [ ]:
df_pairs = df_pairs.drop_duplicates(subset=['hash_id']).reset_index(drop=True)
f"# of UNIQUE GUNWs: {df_pairs.shape[0]}"

# Viewing GUNW pairs

In [ ]:
# start index
M = 0
# number of pairs to view
N = 5

for J in range(M, M + N):
    pair = ifg_pairs[J]

    fig, axs = plt.subplots(1, 2, sharey=True, sharex=True)

    df_ref_plot = pair['reference']
    df_sec_plot = pair['secondary']

    df_ref_plot.plot(column='start_date_str', 
                     legend=True, 
                     ax=axs[0], alpha=.15)
    df_aoi.exterior.plot(ax=axs[0], alpha=.5, color='black')
    axs[0].set_title('Reference')

    df_sec_plot.plot(column='start_date_str', 
                     legend=True, 
                     ax=axs[1], alpha=.15)
    df_aoi.exterior.plot(ax=axs[1], alpha=.5, color='black')
    
    axs[0].set_title(f'Reference {J}')
    axs[1].set_title('Secondary')

# Update types for Graphical Analysis

We want to do some basic visualization to support the understanding if we traverse time correctly. We do some simple standard pandas manipulation.

In [ ]:
df_pairs['reference_date'] = pd.to_datetime(df_pairs['reference_date'])
df_pairs['secondary_date'] = pd.to_datetime(df_pairs['secondary_date'])
df_pairs.head()

# Visualize a Date Graph from Time Series

We can put this into a network Directed Graph and use some simple network functions to check connectivity.

We are going to use just dates for nodes, though you could use `(ref_date, hash_id)` for nodes and then inspect connected components. That is for another notebook.

In [ ]:
# Get unique dates
unique_dates = df_pairs.reference_date.tolist() + df_pairs.secondary_date.tolist()
unique_dates = sorted(list(set(unique_dates)))

# initiate and plot date notes
date2node = {date: k for (k, date) in enumerate(unique_dates)}
node2date = {k: date for (date, k) in date2node.items()}

%matplotlib widget
G = nx.DiGraph()

edges = [(date2node[ref_date], date2node[sec_date]) 
         for (ref_date, sec_date) in zip(df_pairs.reference_date, df_pairs.secondary_date)]
G.add_edges_from(edges)
nx.draw(G)

This function checks there is a path from the first date to the last one. The y-axis is created purely for display so doesn't really indicated anything but flow by month.

In [ ]:
nx.has_path(G, 
            target=date2node[unique_dates[0]],
            source=date2node[unique_dates[-1]])

Ensure that the result above returns a ```True``` value to be able to produce a time-series.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

increment = [date.month + date.day for date in unique_dates]

# source: https://stackoverflow.com/a/27852570
scat = ax.scatter(unique_dates, increment)
position = scat.get_offsets().data

pos = {date2node[date]: position[k] for (k, date) in enumerate(unique_dates)}
nx.draw_networkx_edges(G, pos=pos, ax=ax)
ax.grid('on')
ax.tick_params(axis='x',
               which='major',
               labelbottom=True,
               labelleft=True)
ymin, ymax = ax.get_ylim()

# Deduplication Pt. 2

This is to ensure that previous processing hasn't generate any of the products we have just enumerated.


# Check CMR

This function checks the ASF DAAC if there are GUNWs with the same spatial extent and same date pairs as the ones created. At some point, we will be able to check the input SLC ids from CMR, but currently that is not possible.

If you are processing a new AOI whose products have not been delivered, you can ignore this step. It is a bit time consuming as the queries are done product by product.

In [ ]:
import json
import xml.etree.ElementTree as ET

import requests

COLLECTION_CONCEPT_ID = 'C1595422627-ASF'
CMR_URL = 'https://cmr.earthdata.nasa.gov/search/granules.echo10'

def parse_echo10(echo10_xml: str):
    granules = []
    root = ET.fromstring(echo10_xml)
    for granule in root.findall('result/Granule'):
        g = {
            'product_id': granule.find('GranuleUR').text,
            'product_version': granule.find('GranuleUR').text.split('-')[-1],
            'reference_scenes': [],
            'secondary_scenes': []
        }
        for input_granule in granule.findall('InputGranules/InputGranule'):
            input_granule_type, input_granule_name = input_granule.text.split(' ')
            if input_granule_type == '[Reference]':
                g['reference_scenes'].append(input_granule_name)
            else:
                g['secondary_scenes'].append(input_granule_name)
        granules.append(g)
    return granules


def get_cmr_products(path: int = None):
    session = requests.Session()
    search_params = {
        'provider': 'ASF',
        'collection_concept_id': COLLECTION_CONCEPT_ID,
        'page_size': 2000,
    }
    if path is not None:
        search_params['attribute[]'] = f'int,PATH_NUMBER,{path}'
    headers = {}
    products = []

    while True:
        response = session.get(CMR_URL, params=search_params, headers=headers)
        response.raise_for_status()

        parsed_results = parse_echo10(response.text)
        products.extend(parsed_results)

        if 'CMR-Search-After' not in response.headers:
            break
        headers = {'CMR-Search-After': response.headers['CMR-Search-After']}

    return products

In [ ]:
# query CMR for all existing products in path
results = []
results = get_cmr_products(path_numbers[0])

# sort with descending product version numbers
results = sorted(results, key=lambda d: d['product_version'], reverse = True)

In [ ]:
# convert CMR results to dataframe with the latest product version
new_results = []
track_scenes = []
for i in results:
    ifg_append = i['reference_scenes'] + i['secondary_scenes']
    # pass only first instance of scene combo
    if ifg_append not in track_scenes:
        track_scenes.append(ifg_append)
        new_results.append(i)

results = pd.DataFrame(new_results)
# update column names for merging
results.rename(columns={"reference_scenes": "reference", "secondary_scenes": "secondary"}, inplace = True)

Capture products in CMR

In [ ]:
def capture_cmr_products(row, cmr_products):
    '''Capture products that exist in CMR based on reference and secondary scenes'''
    # concatenate reference and secondary scene in each dataframe row
    row_scenes = row['reference'] + row['secondary']
    # flag True if scenes from enumerator results are within CMR results
    if any(set(row_scenes).issubset(set(x)) for x in cmr_products):
        product_on_cmr = True
    else:
        product_on_cmr = np.nan
    return product_on_cmr

In [ ]:
# Finally filter out pairs in CMR
try:
    # parse all reference and secondary products for each corresponding product in CMR
    cmr_products = results['reference'] + results['secondary']
    cmr_products = cmr_products.to_list()
    # determine which products in the enumerator already exist in CMR
    df_pairs['product_id'] = df_pairs.apply(lambda r: capture_cmr_products(r, cmr_products), axis=1)
    # filter out pairs in CMR
    total_existing_gunws = len(df_pairs[df_pairs['product_id'].notna()])
    print('existing_gunws: ', total_existing_gunws)
    print('Total pairs', df_pairs.shape[0])
    df_pairs_filtered = df_pairs[~df_pairs['product_id'].notna()].reset_index(drop=True)
    df_pairs_filtered.drop_duplicates(subset=['hash_id'], inplace=True)
    print('after filtering, total pairs: ', df_pairs_filtered.shape[0])
except KeyError:
    df_pairs_filtered = copy.deepcopy(df_pairs)
    df_pairs_filtered.drop_duplicates(subset=['hash_id'], inplace=True)
    print('after filtering, total pairs: ', df_pairs_filtered.shape[0])

In [ ]:
if len(df_pairs_filtered) == 0:
    raise Exception('All queried pairs are in CMR, there is nothing to process with specified parameters.')

# Check Hyp3 Account

We are now going to check

1. check products in the open s3 bucket
2. check running/pending jobs

Notes:

1. Above, to accomplish step 1., there is some verbose code (see below). Once we automate delivery, this step will be obsolete. However, until we have delivery, we have to make sure that there are no existing products. Additionally, if we are using a separate (non-operational account), then would be good to use this.
2. If we are debugging products and some of our previously generated products were made incorrectly, we will want to ignore this step.

In [ ]:
# uses .netrc; add `prompt=True` to prompt for credentials; 
hyp3_isce = hyp3_sdk.HyP3(deploy_url)
pending_jobs = hyp3_isce.find_jobs(status_code='PENDING') +  hyp3_isce.find_jobs(status_code='RUNNING')
all_jobs = hyp3_isce.find_jobs()

In [ ]:
print(all_jobs)

## 1. Get existing products in s3 bucket

Get bucket (there is only one)

In [ ]:
from botocore import UNSIGNED
from botocore.config import Config

job_data = [j.to_dict() for j in all_jobs]
job_data_s3 = list(filter(lambda job: 'files' in job.keys(), job_data))
# Only proceed if there are existing jobs
if job_data_s3 != []:
    bucket = job_data_s3[0]['files'][0]['s3']['bucket']

    # Get all keys
    job_keys = [job['files'][0]['s3']['key'] for job in job_data_s3]

    s3 = boto3.resource('s3',config=Config(signature_version=UNSIGNED))
    prod_bucket = s3.Bucket(bucket)

    objects = list(prod_bucket.objects.all())
    ncs = list(filter(lambda x: x.key.endswith('.nc'), objects))

    # Need to physically check if the products are not there (could have been deleted!)

    nc_keys = [nc_ob.key for nc_ob in ncs]
    jobs_with_prods_in_s3 = [job for (k, job) in enumerate(job_data_s3) if job_keys[k] in nc_keys]

    slcs = [(job['job_parameters']['granules'],
            job['job_parameters']['secondary_granules']) 
            for job in jobs_with_prods_in_s3]

    hash_ids_of_prods_in_s3 = [get_gunw_hash_id(*slc) for slc in slcs]

    f"We are removing {df_pairs_filtered['hash_id'].isin(hash_ids_of_prods_in_s3).sum()} GUNWs for submission"

    items = hash_ids_of_prods_in_s3
    df_pairs_filtered = df_pairs_filtered[~df_pairs_filtered['hash_id'].isin(items)].reset_index(drop=True)
    f"Current # of GUNWs: {df_pairs_filtered.shape[0]}"

## 2. Running or Pending Jobs

In [ ]:
# Only proceed if there are existing jobs
if job_data_s3 != []:
    pending_job_data = [j.to_dict() for j in pending_jobs]
    pending_slcs = [(job['job_parameters']['granules'],
                    job['job_parameters']['secondary_granules']) 
                    for job in pending_job_data]

    hash_ids_of_pending_jobs = [get_gunw_hash_id(*slc) for slc in pending_slcs]

    items = hash_ids_of_pending_jobs
    f"We are removing {df_pairs_filtered['hash_id'].isin(items).sum()} GUNWs for submission"

    items = hash_ids_of_pending_jobs
    df_pairs_filtered = df_pairs_filtered[~df_pairs_filtered['hash_id'].isin(items)].reset_index(drop=True)
    f"Current # of GUNWs: {df_pairs_filtered.shape[0]}"

# Visualize a Date Graph from the Final Filtered Time Series

We can put this into a network Directed Graph and use some simple network functions to check connectivity (*which may not be applicable).

We are going to use just dates for nodes, though you could use `(ref_date, hash_id)` for nodes and then inspect connected components. That is for another notebook.

In [ ]:
# Get unique dates
unique_dates = df_pairs_filtered.reference_date.tolist() + df_pairs_filtered.secondary_date.tolist()
unique_dates = sorted(list(set(unique_dates)))

# initiate and plot date notes
date2node = {date: k for (k, date) in enumerate(unique_dates)}
node2date = {k: date for (date, k) in date2node.items()}

%matplotlib widget
G = nx.DiGraph()

edges = [(date2node[ref_date], date2node[sec_date]) 
         for (ref_date, sec_date) in zip(df_pairs_filtered.reference_date, df_pairs_filtered.secondary_date)]
G.add_edges_from(edges)
nx.draw(G)

This function checks there is a path from the first date to the last one. The y-axis is created purely for display so doesn't really indicated anything but flow by month.

*Again, this may not be applicable in cases where parts of the network had already been deployed before and/or you are densifying by specifying temporal sampling.
In such cases, these plots serve merely as a sanity check.

In [ ]:
nx.has_path(G, 
            target=date2node[unique_dates[0]],
            source=date2node[unique_dates[-1]])

Ensure that the result above returns a ```True``` value to be able to produce a time-series.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

increment = [date.month + date.day for date in unique_dates]

# source: https://stackoverflow.com/a/27852570
scat = ax.scatter(unique_dates, increment)
position = scat.get_offsets().data

pos = {date2node[date]: position[k] for (k, date) in enumerate(unique_dates)}
nx.draw_networkx_edges(G, pos=pos, ax=ax)
ax.grid('on')
ax.tick_params(axis='x',
               which='major',
               labelbottom=True,
               labelleft=True)
ymin, ymax = ax.get_ylim()

# Submit jobs to Hyp3

In [ ]:
records_to_submit = df_pairs_filtered.to_dict('records')
records_to_submit[0]

The below puts the records in a format that we can submit to the Hyp3 API.

**Note 1**: there is an index in the records to submit to ensure we don't over submit jobs for generating GUNWs. \
**Note 2**: uncomment the code to *actually* submit the jobs.

In [ ]:
# uses .netrc; add `prompt=True` to prompt for credentials; 
hyp3_isce = hyp3_sdk.HyP3(deploy_url)

# NOTE: we are using "INSAR_ISCE" for the `main` branch.
# Change this to "INSAR_ISCE_TEST" to use the `dev` branch, but ONLY if you know what you're doing
# chaging to dev will overwrite the product version number and make it difficult to dedup
job_type = 'INSAR_ISCE'

job_dicts = [{'name': job_name,
              'job_type': job_type,
              'job_parameters': {'granules': r['reference'],
                                 'secondary_granules': r['secondary']}} 
             # NOTE THERE IS AN INDEX - this is to submit only a subset of Jobs
             for r in records_to_submit]

In [ ]:
# Report summary of all job parameters
print("Start date is '{}'".format(unique_dates[0]))
print("End date is '{}'".format(unique_dates[-1]))
print("GUNWs expected '{}'".format(len(job_dicts)))
print("Job Name is '{}'".format(job_name))
print("Shapefile Name is '{}'".format(aoi_shapefile.split('/')[-1]))

In [ ]:
#UNCOMMENT TO SUBMIT
#prepared_jobs = job_dicts
#submitted_jobs = hyp3_sdk.Batch()
#for batch in hyp3_sdk.util.chunk(prepared_jobs):
#    submitted_jobs += hyp3_isce.submit_prepared_jobs(batch)

Query all jobs on the server

In [ ]:
jobs = hyp3_isce.find_jobs()
print(jobs)

Query your particular job

In [ ]:
jobs = hyp3_isce.find_jobs(name=job_name)
print(jobs)

In [ ]:
# # create clean directory to deposit products in
if os.path.exists(prod_dir):
    os.remove(prod_dir)

os.mkdir(prod_dir)

Below, we show how to download files. The multi-threading example will download products in parallel much faster than `jobs.download_files()`.

In [ ]:
jobs = hyp3_isce.find_jobs(name=job_name)
print(jobs)

import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(tqdm(executor.map(lambda job: job.download_files(), jobs), total=len(jobs)))